# **Etapa 1**

In [22]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

spark = SparkSession.builder.master("local[*]").appName("Exercicio Intro").getOrCreate()

df_nomes = spark.read.csv("nomes_aleatorios.txt")

df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



# **Etapa 2**

In [23]:
df_nomes.printSchema()

df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")

df_nomes.show(10)

root
 |-- _c0: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



# **Etapa 3**

In [24]:
from pyspark.sql.functions import when, rand

df_nomes = df_nomes = df_nomes.withColumn("Escolaridade", when(rand() < 0.33, "Fundamental").when(rand() > 0.66, "Médio").otherwise("Superior"))

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet|       Médio|
|    Jamie Russell|       Médio|
|   Edward Kistler|    Superior|
|    Sheila Maurer|    Superior|
| Donald Golightly|    Superior|
|       David Gray| Fundamental|
|      Joy Bennett|    Superior|
|      Paul Kriese|       Médio|
|Berniece Ornellas| Fundamental|
|    Brian Farrell|       Médio|
+-----------------+------------+
only showing top 10 rows



# **Etapa 4**

In [25]:
paises = [
    "Argentina", "Bolívia", "Brasil", "Chile", "Colômbia", "Ecuador", 
    "Guiana", "Paraguai", "Peru", "Suriname", "Uruguai", "Venezuela", "Guiana Francesa"
]

df_nomes = df_nomes.withColumn(
    "Pais",
    when(rand() < 1/13, paises[0])
    .when(rand() < 2/13, paises[1])
    .when(rand() < 3/13, paises[2])
    .when(rand() < 4/13, paises[3])
    .when(rand() < 5/13, paises[4])
    .when(rand() < 6/13, paises[5])
    .when(rand() < 7/13, paises[6])
    .when(rand() < 8/13, paises[7])
    .when(rand() < 9/13, paises[8])
    .when(rand() < 10/13, paises[9])
    .when(rand() < 11/13, paises[10])
    .when(rand() < 12/13, paises[11])
    .otherwise(paises[12])
)

df_nomes.show(10)

+-----------------+------------+---------+
|            Nomes|Escolaridade|     Pais|
+-----------------+------------+---------+
|   Frances Bennet|       Médio|  Bolívia|
|    Jamie Russell|       Médio|  Bolívia|
|   Edward Kistler|    Superior| Paraguai|
|    Sheila Maurer|    Superior|Argentina|
| Donald Golightly|    Superior|  Uruguai|
|       David Gray| Fundamental|   Brasil|
|      Joy Bennett|    Superior|    Chile|
|      Paul Kriese|       Médio| Colômbia|
|Berniece Ornellas| Fundamental|  Bolívia|
|    Brian Farrell|       Médio|    Chile|
+-----------------+------------+---------+
only showing top 10 rows



# **Etapa 5**

In [26]:
ano_inicio = 1945
ano_fim = 2010

df_nomes = df_nomes.withColumn("Ano", (rand() * (ano_fim - ano_inicio + 1) + ano_inicio).cast("int"))

df_nomes.show(10)

+-----------------+------------+---------+----+
|            Nomes|Escolaridade|     Pais| Ano|
+-----------------+------------+---------+----+
|   Frances Bennet|       Médio|  Bolívia|1955|
|    Jamie Russell|       Médio|  Bolívia|1988|
|   Edward Kistler|    Superior| Paraguai|1948|
|    Sheila Maurer|    Superior|Argentina|1958|
| Donald Golightly|    Superior|  Uruguai|1984|
|       David Gray| Fundamental|   Brasil|2010|
|      Joy Bennett|    Superior|    Chile|1960|
|      Paul Kriese|       Médio| Colômbia|1974|
|Berniece Ornellas| Fundamental|  Bolívia|2000|
|    Brian Farrell|       Médio|    Chile|1974|
+-----------------+------------+---------+----+
only showing top 10 rows



# **Etapa 6**

In [27]:
from pyspark.sql.functions import col

df_select = df_nomes.select("Nomes").where(col('Ano') > 2000)

df_select.show(10)

+---------------+
|          Nomes|
+---------------+
|     David Gray|
|   David Medina|
|       Mary Lee|
| Wilfredo Grant|
|  Michael Agnew|
|    Donald Vogt|
|  Milton Dillon|
|    Milton Rowe|
|Charles Randall|
|  Evelyn Shaver|
+---------------+
only showing top 10 rows



# **Etapa 7**

In [28]:
df_nomes.createOrReplaceGlobalTempView("pessoas")

spark.sql("SELECT Nomes FROM global_temp.pessoas WHERE Ano > 2000").show()


+---------------+
|          Nomes|
+---------------+
|     David Gray|
|   David Medina|
|       Mary Lee|
| Wilfredo Grant|
|  Michael Agnew|
|    Donald Vogt|
|  Milton Dillon|
|    Milton Rowe|
|Charles Randall|
|  Evelyn Shaver|
|  Joyce Bennett|
|    Linda Batts|
|    John Snipes|
|    Wendi Klein|
|    Velma Davis|
|   Frank France|
|  Ella Mitchell|
|    Danny Bloom|
|      Ann Scott|
|  Margaret Lacy|
+---------------+
only showing top 20 rows



# **Etapa 8**

In [29]:
df_nomes.filter((col("Ano") > 1979) & (col("Ano") < 1995)).count()


2272267

# **Etapa 9**

In [30]:
spark.sql("SELECT count(*) as Millennials FROM global_temp.pessoas WHERE Ano > 1979 AND Ano < 1995").show()

+-----------+
|Millennials|
+-----------+
|    2272267|
+-----------+



# **Etapa 10**

In [32]:
df_resultado = spark.sql("""
    SELECT 
        Pais,
        CASE
            WHEN Ano < 1965 THEN 'Baby Boomers'
            WHEN Ano < 1980 THEN 'Geração X'
            WHEN Ano < 1995 THEN 'Millennials (Geração Y)'
            ELSE 'Geração Z'
        END AS Geracao,  
        COUNT(*) AS Quantidade
    FROM global_temp.pessoas
    GROUP BY Pais, 
        CASE 
            WHEN Ano < 1965 THEN 'Baby Boomers'
            WHEN Ano < 1980 THEN 'Geração X'
            WHEN Ano < 1995 THEN 'Millennials (Geração Y)'
            ELSE 'Geração Z'
        END
    ORDER BY Pais, Quantidade, 
        CASE 
            WHEN Ano < 1965 THEN 'Baby Boomers'
            WHEN Ano < 1980 THEN 'Geração X'
            WHEN Ano < 1995 THEN 'Millennials (Geração Y)'
            ELSE 'Geração Z'
        END
""")

df_resultado.show(52)


+---------------+--------------------+----------+
|           Pais|             Geracao|Quantidade|
+---------------+--------------------+----------+
|      Argentina|           Geração X|    174260|
|      Argentina|Millennials (Gera...|    174729|
|      Argentina|           Geração Z|    186741|
|      Argentina|        Baby Boomers|    234092|
|        Bolívia|           Geração X|    322717|
|        Bolívia|Millennials (Gera...|    322870|
|        Bolívia|           Geração Z|    343978|
|        Bolívia|        Baby Boomers|    430503|
|         Brasil|           Geração X|    409339|
|         Brasil|Millennials (Gera...|    409652|
|         Brasil|           Geração Z|    436881|
|         Brasil|        Baby Boomers|    545438|
|          Chile|           Geração X|    419738|
|          Chile|Millennials (Gera...|    419879|
|          Chile|           Geração Z|    449067|
|          Chile|        Baby Boomers|    560353|
|       Colômbia|           Geração X|    363811|
